# Installing DEAP


Assuming that your other package versions are compatible with DEAP:

1. Clone the repo https://github.com/DEAP/deap.git

2. Open a command window in the folder where the DEAP setup.py file is located and type the following: 

        conda install -c conda-forge pypandoc
        python setup.py install



In [2]:
import matplotlib.pyplot as plt
import sys
import array
import random
import numpy as np

from deap import  algorithms
from deap import  base
from deap import  creator
from deap import  tools

%matplotlib inline

## Logbook to Pandas DataFrame

Logbook to pandas for multistats with chapters.  


The DEAP log object has fitness statistics for every generation. This is what is printed out during a verbose evolutionary run. The log object is not a simple data structure, hence the `logtoDF` function is used to extract the information into a Pandas dataframe. The data frame columns `gen` and `nevals` are scalar values describing the generation number and the number of evaluations in this generation.  The remaining columns `avg`, `std`, `min`, and  `max` contain numpy arrays with value for each of the objectives in this multi-objective problem.  In this form the data is not easily accessible and another function `explode` is used to extract the array columns into rows in the data frame, adding a new column `ridx` that acts as a row index.  In this new form we can filter the dataframe and extract only the data for a specific objective.


https://stackoverflow.com/questions/42904904/saving-deap-results-into-pandas-dataframe  
https://stackoverflow.com/questions/27263805/pandas-when-cell-contents-are-lists-create-a-row-for-each-element-in-the-list      
https://stackoverflow.com/questions/12680754/split-explode-pandas-dataframe-string-entry-to-separate-rows/40449726#40449726  

In [3]:
import pandas as pd
from functools import reduce
from operator import add, itemgetter

def logtoDF(log):
    df = None
    chapter_keys = log.chapters.keys()
    if chapter_keys:
        sub_chaper_keys = [c[0].keys() for c in log.chapters.values()]


        data = [list(map(itemgetter(*skey), chapter)) for skey, chapter in zip(sub_chaper_keys, log.chapters.values())]
        data = np.array([[*a, *b] for a, b in zip(*data)])

        columns = reduce(add, [["_".join([x, y]) for y in s] for x, s in zip(chapter_keys, sub_chaper_keys)])
        df = pd.DataFrame(data, columns=columns)

        keys = log[0].keys()
        data = [[d[k] for d in logbook] for k in keys]
        for d, k in zip(data, keys):
            df[k] = d
    else:
        data = [[i for i in item.values()] for item in log]
        df = pd.DataFrame(data, columns=log.header)
    
    return df


        

In [ ]:
# to expands np.arrays in columns, creating new rows for each set of array entries
def explode(df, lst_cols, fill_value='', preserve_index=False):
    # make sure `lst_cols` is list-alike
    if (lst_cols is not None
        and len(lst_cols) > 0
        and not isinstance(lst_cols, (list, tuple, np.ndarray, pd.Series))):
        lst_cols = [lst_cols]
    # all columns except `lst_cols`
    idx_cols = df.columns.difference(lst_cols)
    # calculate lengths of lists
    lens = df[lst_cols[0]].str.len()
    # preserve original index values    
    idx = np.repeat(df.index.values, lens)
    # create "exploded" DF
    res = (pd.DataFrame({
                col:np.repeat(df[col].values, lens)
                for col in idx_cols},
                index=idx)
             .assign(**{col:np.concatenate(df.loc[lens>0, col].values)
                            for col in lst_cols}))
    # append those rows that have empty lists
    if (lens == 0).any():
        # at least one list in cells is empty
        res = (res.append(df.loc[lens==0, idx_cols], sort=False)
                  .fillna(fill_value))
    # revert the original index order
    res = res.sort_index()
    # reset index if requested
    if not preserve_index:        
        res = res.reset_index(drop=True)
    return res

## Attaching names to the cities

https://stackoverflow.com/questions/47429746/how-to-create-list-of-city-as-individual-in-deap-python

In [4]:
def generate_individual():
    return ["Almeria","Cadiz","Cordoba","Granada","Huelva","Jaen","Malaga", "Seville"]

creator.create("FitnessMin", base.Fitness,weights=(-1.0,))
creator.create("Individual",list, fitness=creator.FitnessMin)
toolbox = base.Toolbox()
toolbox.register("indices", generate_individual) # Gen, in this case, a number which represents a city.
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.indices) # Define a route of cities. A chromosome.
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
POP_SIZE = 50
pop = toolbox.population(n=POP_SIZE)
print(pop)

[['Almeria', 'Cadiz', 'Cordoba', 'Granada', 'Huelva', 'Jaen', 'Malaga', 'Seville'], ['Almeria', 'Cadiz', 'Cordoba', 'Granada', 'Huelva', 'Jaen', 'Malaga', 'Seville'], ['Almeria', 'Cadiz', 'Cordoba', 'Granada', 'Huelva', 'Jaen', 'Malaga', 'Seville'], ['Almeria', 'Cadiz', 'Cordoba', 'Granada', 'Huelva', 'Jaen', 'Malaga', 'Seville'], ['Almeria', 'Cadiz', 'Cordoba', 'Granada', 'Huelva', 'Jaen', 'Malaga', 'Seville'], ['Almeria', 'Cadiz', 'Cordoba', 'Granada', 'Huelva', 'Jaen', 'Malaga', 'Seville'], ['Almeria', 'Cadiz', 'Cordoba', 'Granada', 'Huelva', 'Jaen', 'Malaga', 'Seville'], ['Almeria', 'Cadiz', 'Cordoba', 'Granada', 'Huelva', 'Jaen', 'Malaga', 'Seville'], ['Almeria', 'Cadiz', 'Cordoba', 'Granada', 'Huelva', 'Jaen', 'Malaga', 'Seville'], ['Almeria', 'Cadiz', 'Cordoba', 'Granada', 'Huelva', 'Jaen', 'Malaga', 'Seville'], ['Almeria', 'Cadiz', 'Cordoba', 'Granada', 'Huelva', 'Jaen', 'Malaga', 'Seville'], ['Almeria', 'Cadiz', 'Cordoba', 'Granada', 'Huelva', 'Jaen', 'Malaga', 'Seville'], ['A

In [8]:
list(range(len(pop)))

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49]

In [ ]:
## Print the values of a population

In [16]:
# to print the population
def printPop(pop,start=0,end=None):
    print(60*'=')
    if end is None:
        end = len(pop)
    print(f'Population size: {len(pop)}')
    lst = list(range(len(pop)))
    for ip in lst[start:end]:
        indv = pop[ip]
        print(60*'-')
        print(f'\npopulation[{ip}]:') 
        for ii in range(len(indv)):
            print(f'{indv[ii]}') 
    print(60*'=')

printPop(pop,0,5)

Population size: 50
------------------------------------------------------------

population[0]:
Almeria
Cadiz
Cordoba
Granada
Huelva
Jaen
Malaga
Seville
------------------------------------------------------------

population[1]:
Almeria
Cadiz
Cordoba
Granada
Huelva
Jaen
Malaga
Seville
------------------------------------------------------------

population[2]:
Almeria
Cadiz
Cordoba
Granada
Huelva
Jaen
Malaga
Seville
------------------------------------------------------------

population[3]:
Almeria
Cadiz
Cordoba
Granada
Huelva
Jaen
Malaga
Seville
------------------------------------------------------------

population[4]:
Almeria
Cadiz
Cordoba
Granada
Huelva
Jaen
Malaga
Seville
------------------------------------------------------------

population[5]:
Almeria
Cadiz
Cordoba
Granada
Huelva
Jaen
Malaga
Seville
------------------------------------------------------------

population[6]:
Almeria
Cadiz
Cordoba
Granada
Huelva
Jaen
Malaga
Seville
------------------------------------------